In [ ]:
import re
import pandas as pd

import numpy as np

from tqdm import tqdm
import os

tqdm.pandas(desc='pandas bar')

pd.set_option('max_columns', None)
pd.set_option('max_rows', 300)
pd.set_option('float_format', lambda x: '%.3f' % x)

%matplotlib inline

data_dir_s1 = './DATA/s1/'
data_dir_s2 = './DATA/s2/'
data_dir_s3 = './DATA/s3/'

data_dir_t1 = './DATA/t1/'
data_dir_t2 = './DATA/t2/'

In [ ]:

# # source市场
# train_s1 = pd.read_table(data_dir_s1+'train.tsv')
# train_s2 = pd.read_table(data_dir_s2+'train.tsv')
# train_s3 = pd.read_table(data_dir_s3+'train.tsv')

# train_core_s1 = pd.read_table(data_dir_s1+'train_5core.tsv')
# train_core_s2 = pd.read_table(data_dir_s2+'train_5core.tsv')
# train_core_s3 = pd.read_table(data_dir_s3+'train_5core.tsv')

# target市场
train_5coret1 = pd.read_table(data_dir_t1+'train_5core.tsv')
train_t1 = pd.read_table(data_dir_t1+'train.tsv')
vaild_qrel_t1 = pd.read_table(data_dir_t1+'valid_qrel.tsv')
vaild_pred_t1 = pd.read_table('/ljw/wsdm22_cup_xmrec/wsdm22_cup_xmrec_rate/baseline_outputs/sample_run/t1/valid_pred.tsv')
test_t1 = pd.read_table(data_dir_t1+'test_run.tsv')

train_5coret2 = pd.read_table(data_dir_t2+'train_5core.tsv')
train_t2 = pd.read_table(data_dir_t2+'train.tsv')
vaild_qrel_t2 = pd.read_table(data_dir_t2+'valid_qrel.tsv')
vaild_pred_t2 = pd.read_table('/ljw/wsdm22_cup_xmrec/wsdm22_cup_xmrec_rate/baseline_outputs/sample_run/t2/valid_pred.tsv')

In [ ]:
train_cross_t1 = train_5coret1.merge(train_t1, on=['userId', 'itemId'], how='left')
train_5coret1 = train_cross_t1[train_cross_t1['rating_y'].isnull()][['userId', 'itemId']]  

train_t1 = pd.concat([train_t1, train_5coret1, train_5coret1, train_5coret1, train_5coret1, train_5coret1], ignore_index=True)
user_feature_list = train_t1.groupby("userId").agg(rating_list=("rating", list)).reset_index()
user_feature_list['item_num'] = user_feature_list['rating_list'].apply(len)

In [ ]:
vaild_pred_t1.head(2)

In [ ]:
vaild_qrel_t1.head(2)

In [ ]:
# t1
vaild = vaild_pred_t1.merge(vaild_qrel_t1, on=['userId', 'itemId'], how='left')
vaild['score'] = round(vaild['score'], 4)
vaild = vaild.groupby("userId").agg(
    score_list=("score", list),
    rating_pos=("rating", list)  # 有rating代表预测的是这个位置
).reset_index()

vaild = vaild.merge(user_feature_list[['userId','item_num','rating_list']], on='userId', how='left')

vaild['score_std'] = vaild['score_list'].apply(lambda x:np.std(x[:10]))
# 找到预测的位置
vaild['posi_pos'] = vaild['rating_pos'].apply(lambda x:x.index(1.0)+1)
print('t1  all: {},pos of 1st: {},pos of >10: {}'.format(vaild.shape[0], vaild[vaild['posi_pos']==1].shape[0], vaild[vaild['posi_pos']>10].shape[0]))

# # t2
# vaild = vaild_pred_t2.merge(vaild_qrel_t2, on=['userId', 'itemId'], how='left')
# vaild = vaild.groupby("userId").agg(
#     score_list=("score", list),
#     rating_list=("rating", list)
# ).reset_index()

# # 找到预测的位置
# vaild['posi_pos'] = vaild['rating_list'].apply(lambda x:x.index(1.0)+1)
# print('t2  all: {},pos of 1st: {},pos of >10: {}'.format(vaild.shape[0], vaild[vaild['posi_pos']==1].shape[0], vaild[vaild['posi_pos']>10].shape[0]))

In [ ]:
vaild.head(300)

In [ ]:
print("x>10 pred score std'max:", vaild[vaild['posi_pos']>10]['score_std'].max())
print("x>10 pred score std'min:", vaild[vaild['posi_pos']>10]['score_std'].min())

print("1<x<=10 pred score std'max:", vaild[(vaild['posi_pos']>1)&(vaild['posi_pos']<=10)]['score_std'].max())
print("1<x<=10 pred score std'min:", vaild[(vaild['posi_pos']>1)&(vaild['posi_pos']<=10)]['score_std'].min())

print("x==1 pred score std'max:", vaild[vaild['posi_pos']==1]['score_std'].max())
print("x==1 pred score std'min:", vaild[vaild['posi_pos']==1]['score_std'].min())